In [58]:
from policyengine_us import Simulation
from policyengine_core.reforms import Reform
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from policyengine_core.charts import *


In [42]:
reform_ref = Reform.from_dict({
  "gov.irs.credits.ctc.amount.base[0].amount": {
    "2024-01-01.2100-12-31": 5000
  },
  "gov.irs.credits.ctc.phase_out.amount": {
    "2024-01-01.2100-12-31": 0
  },
  "gov.irs.credits.ctc.refundable.fully_refundable": {
    "2024-01-01.2100-12-31": True
  }
}, country_id="us")

reform_non_ref = Reform.from_dict({
  "gov.irs.credits.ctc.amount.base[0].amount": {
    "2024-01-01.2100-12-31": 5000
  },
  "gov.irs.credits.ctc.phase_out.amount": {
    "2024-01-01.2100-12-31": 0
  }
}, country_id="us")

In [43]:
joint_situation = {
  "people": {
    "you": {
      "age": {
        "2025": 40
      }
    },
    "your partner": {
      "age": {
        "2025": 40
      }
    },
    "your first dependent": {
      "age": {
        "2025": 10
      }
    }
  },
  "families": {
    "your family": {
      "members": [
        "you",
        "your partner",
        "your first dependent"
      ]
    }
  },
  "marital_units": {
    "your marital unit": {
      "members": [
        "you",
        "your partner"
      ]
    },
    "your first dependent's marital unit": {
      "members": [
        "your first dependent"
      ],
      "marital_unit_id": {
        "2025": 1
      }
    }
  },
  "tax_units": {
    "your tax unit": {
      "members": [
        "you",
        "your partner",
        "your first dependent"
      ]
    }
  },
  "spm_units": {
    "your household": {
      "members": [
        "you",
        "your partner",
        "your first dependent"
      ]
    }
  },
  "households": {
    "your household": {
      "members": [
        "you",
        "your partner",
        "your first dependent"
      ],
      "state_name": {
        "2025": "TX"
      }
    }
  },
  "axes": [
    [
      {
        "name": "employment_income",
        "count": 200,
        "min": 0,
        "max": 500000
      }
    ]
  ]
}




In [44]:
hoh_situation = {
  "people": {
    "you": {
      "age": {
        "2025": 40
      }
    },
    "your first dependent": {
      "age": {
        "2025": 10
      }
    }
  },
  "families": {
    "your family": {
      "members": [
        "you",
        "your first dependent"
      ]
    }
  },
  "marital_units": {
    "your marital unit": {
      "members": [
        "you"
      ]
    },
    "your first dependent's marital unit": {
      "members": [
        "your first dependent"
      ],
      "marital_unit_id": {
        "2025": 1
      }
    }
  },
  "tax_units": {
    "your tax unit": {
      "members": [
        "you",
        "your first dependent"
      ]
    }
  },
  "spm_units": {
    "your household": {
      "members": [
        "you",
        "your first dependent"
      ]
    }
  },
  "households": {
    "your household": {
      "members": [
        "you",
        "your first dependent"
      ],
      "state_name": {
        "2025": "TX"
      }
    }
  },
  "axes": [
    [
      {
        "name": "employment_income",
        "count": 200,
        "min": 0,
        "max": 500000
      }
    ]
  ]
}




In [45]:
joint_baseline = Simulation(
    situation=joint_situation,
)


joint_non_ref_simulation = Simulation(
    reform=reform_non_ref,
    situation=joint_situation,
)

joint_ref_simulation = Simulation(
    reform=reform_ref,
    situation=joint_situation,
)


In [46]:
hoh_baseline = Simulation(
    situation=hoh_situation,
)


hoh_non_ref_simulation = Simulation(
    reform=reform_non_ref,
    situation=hoh_situation,
)

hoh_ref_simulation = Simulation(
    reform=reform_ref,
    situation=hoh_situation,
)

In [47]:
joint_baseline_income = joint_baseline.calculate("household_net_income", 2025)
joint_non_ref_income = joint_non_ref_simulation.calculate("household_net_income", 2025)
joint_ref_income = joint_ref_simulation.calculate("household_net_income", 2025)

In [48]:
hoh_baseline_income = hoh_baseline.calculate("household_net_income", 2025)
hoh_non_ref_income = hoh_non_ref_simulation.calculate("household_net_income", 2025)
hoh_ref_income = hoh_ref_simulation.calculate("household_net_income", 2025)

In [65]:
def create_income_difference_graph(baseline_income, non_ref_income, ref_income, situation, color):
    # Calculate differences (reform - baseline)
    non_ref_diff = non_ref_income - baseline_income
    ref_diff = ref_income - baseline_income

    # Create traces for each reform
    trace_non_ref = go.Scatter(
        x=baseline_income, 
        y=non_ref_diff, 
        mode='lines', 
        name=f'{situation} Non-Refundable Reform',
        line=dict(color=color)
    )
    trace_ref = go.Scatter(
        x=baseline_income, 
        y=ref_diff, 
        mode='lines', 
        name=f'{situation} Refundable Reform',
        line=dict(color=color, dash='dash')
    )

    return [trace_non_ref, trace_ref]

In [67]:
# Create traces for joint and HOH scenarios
joint_traces = create_income_difference_graph(joint_baseline_income, joint_non_ref_income, joint_ref_income, 'Joint', 'blue')
hoh_traces = create_income_difference_graph(hoh_baseline_income, hoh_non_ref_income, hoh_ref_income, 'HOH', 'red')

# Combine all traces
all_traces = joint_traces + hoh_traces


In [68]:
# Create the layout
layout = go.Layout(
    title='JD Vance increase Child Tax Credit proposal',
    xaxis=dict(
        title='Household Employment Income',
        tickformat='$,.0f',
    ),
    yaxis=dict(
        title='Income Change (Reform - Baseline)',
        tickformat='$,.0f',
    ),
    showlegend=True
)

# Create the figure and add the traces
fig = go.Figure(data=all_traces, layout=layout)

# Add a horizontal line at y=0 for reference
fig.add_hline(y=0, line_dash="dash", line_color="gray")

fig = format_fig(fig)  # Assuming this function exists and applies additional formatting

# Display the graph
fig.show()